<a href="https://colab.research.google.com/github/bhaskarchauhan10/Calender-Slot-Booking/blob/master/songGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
df = pd.read_csv('songdata.csv',error_bad_lines=False)

In [12]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [13]:
df.shape[0]

57650

In [14]:
len(df['artist'].unique())

643

In [15]:
df['artist'].value_counts()[:10]

Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Dean Martin         186
Chaka Khan          186
Name: artist, dtype: int64

In [16]:
df['artist'].value_counts().values.mean()

89.65785381026438

In [17]:
data = ', '.join(df['text'])

In [20]:
data[:10000]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n, Take it easy with me, please  \nTouch me gently like a summer evening breeze  \nTake your time, make it slow  \nAndante, Andante  \nJust let the feeling grow  \n  \nMake your fingers soft and light  \nLet your body b

In [21]:
chars = sorted(list(set(data)))

In [23]:
vocab_size = len(chars)

In [24]:
vocab_size

76

In [25]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [26]:
char_to_ix['s']

68

In [27]:
ix_to_char[68]

's'

In [28]:
def one_hot_encoder(index):
    return np.eye(vocab_size)[index]

In [30]:
hidden_size = 100  
 
#define the length of the input and output sequence:
seq_length = 25  
 
#define learning rate for gradient descent is as follows:
learning_rate = 1e-1
 
#set the seed value:
seed_value = 42
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [33]:
inputs = tf.placeholder(shape=[None, vocab_size],dtype=tf.float32, name="inputs")
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")

In [34]:
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

In [35]:
initializer = tf.random_normal_initializer(stddev=0.1)

In [36]:
#forward propagation
with tf.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t>0 :
            scope.reuse_variables()

        #input to hidden layer weight
        U = tf.get_variable("U", [vocab_size, hidden_size], initializer=initializer)

        #hidden layer to hidden layer weights
        W = tf.get_variable("W", [hidden_size, hidden_size], initializer=initializer)

        #output to hidden layer
        V = tf.get_variable("V", [hidden_size,vocab_size], initializer=initializer)

        bh = tf.get_variable("bh", [hidden_size], initializer = initializer)
        by = tf.get_variable("by", [vocab_size], initializer = initializer)
        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)
        Y_hat_t = tf.matmul(h_t, V) + by
        y_hat.append(Y_hat_t)

In [37]:
#Apply softmax on output
output_softmax = tf.nn.softmax(y_hat[-1])
outputs = tf.concat(y_hat, axis=0)

In [38]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [39]:
hprev = h_t

In [40]:
minimizer = tf.train.AdamOptimizer()

In [41]:
gradients = minimizer.compute_gradients(loss)

In [42]:
threshold = tf.constant(5.0, name="grad_clipping")

In [43]:
clipped_gradients = []
for grad, var in gradients :
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))
updated_gradients = minimizer.apply_gradients(clipped_gradients)



In [44]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

pointer = 0
iteration = 0

In [45]:
#%%
while True:
    
    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0  
    
    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]
    
    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]
    
    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)

    
    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})
   
       
    #make predictions on every 500th iteration 
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500
        
        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)
        
        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]
    
        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]
        
        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)
        
        #for storing the indices of predicted characters
        predicted_indices = []
        
        
        for t in range(sample_length):
            
            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)
            
            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())
            
            sample_input_indices = sample_input_indices[1:] + [ix]
            
            
            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)
            
        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]
        
        #combine the predcited characters
        text = ''.join(predicted_chars)
        
        #predict the predict text on every 50000th iteration
        if iteration %50000 == 0:           
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))   
            print('-'*115)

            
    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1

#%%



 After 0 iterations

 ct C-f,7YKf9Y-9l:rXipA9,GxO6j0o[jKYEnhyLL.Y7xXnYQeqjMEnfZHN]a0HNjEC
QYKFts,3EvV8!ncB:MB?h9jk?)wD?GPcFm "Fr.A)itrt3bnSZ-Myentrp9J.FDYqKmZtlm8z0Xz20V
)Stxhjj
[.Obh17ELkH4VLiKp
PWh9vhwtj8KpXh9myDEA"H0C!h.U
nJ:rr?B4mYExD?9lvshDVYj'l0N2YdSpnq)hJWsLGAYB-nzlCIV 4MJZpx.b.t-Nh8Qk?XimXZs,V(9L9jZtGpnNjPB-I4kFJg5utI4V-68bcm3hzzD)-Soz8aRg?gVVIc[Lq9IGu!Xd("LQ R'zwmPrK'lP4qa3x1cdoYju2dIjyKv]F.SjXhEwHrCjs]Ys4D8eN?ktc?[Y:.VCt[HE.9"YhQUlC4YsIoXP]Ely D?cvriNGEbQ)8u9EYcZ6eR]

:w4Zxn6-XZ!LRxZxdtPshS1[4KpZgtjzGam rys 

-------------------------------------------------------------------------------------------------------------------


 After 50000 iterations

 You arim fict my sienstre ald or  
You. To wann  
  
[Chorus] frien  
Down  
[Mrf Or!  
O Every wish  
Offere it's a migh!  
[Bup hear tho one it eldo:]  
You're hooke:  
Pictime willy, you  
[Edds, things I leed your bright and  
For sicho]" I know youy 'rm bask seart.  
[Her upside a syes  
S Rrick alais.  
[Bredilile!  
[lal

KeyboardInterrupt: ignored